In [1]:
    if __name__ == '__main__':
        %reload_ext pidgin

In [2]:
    import textwrap, typing as t

    import IPython, tokenize, doctest, types, itertools, operator, ast, importnb
    from pidgin import tangle
        
    from pidgin.specifications import testing, transform_cell
    

In [3]:
    def line_row_col(str, int: "Cursor Position"):
        line, offset = IPython.utils.tokenutil.line_at_cursor(str, int)
        assert offset <= int
        row = len(str[:offset].splitlines())+1
        col = int - offset
        return line, row, col 

In [4]:
    def get_example(str, row):
        finder = doctest.DocTestFinder()
        for test in finder.find(str, ''):
            for example in test.examples:
                if example.lineno < row <= (example.lineno + len(example.source.splitlines())): 
                    test.examples = [example]
                    return test

In [5]:
    def get_node(str, row, _node=None):
        for node in ast.parse(str).body:
            if node.lineno-1 > row: return _node
            _node = node
        return node

In [6]:
    def inline_code(line, col):
        open, input = False, ""
        for token in IPython.utils.tokenutil.generate_tokens(__import__('io').StringIO(line).readline):
            if token.end[1] > col:
                if token.string == '`' and open: break
            if token.string == '`': 
                open = not open
            else:
                if open: input += token.string
                else: input = ''
            
        return input

In [7]:
    shell = get_ipython()

In [8]:
    def display_Str(str):
        with shell.weave as tmp: 
            return tmp.run_cell(str)

In [9]:
    def catch(func, *args, **kwargs):
        with IPython.utils.capture.capture_output() as out:
            result = func(*args, **kwargs)
        if out.outputs:
            return out.outputs[0].data
        if out.stdout.strip():
            return {'text/plain': out.stdout}
        
        if out.stderr.strip():
            return {'text/plain': out.stderr}

In [10]:
    def dispatch_inspector(str, int: "Cursor position"):
        line, row, col  = line_row_col(str, int)
        node = get_node(transform_cell.markdown_to_python(str), row)
        shell = IPython.get_ipython()
        if isinstance(node, ast.Expr) and isinstance(node.value, ast.Str):
            if '>>>' in str:
                test = get_example(str, row)
                if test: return catch(testing.run_test, test, shell.test) or {'text/plain': test.examples[0].source}
            inline = inline_code(line, col)
            if inline: return catch(shell.weave.run_literal, inline) or {'text/plain': inline}
            return catch(display_Str, line.strip() and node.value.s or str)
        return None

In [11]:
     def do_inspect(kernel, code, cursor_pos, detail_level=0):
        reply_content = {'status' : 'ok'}
        reply_content['data'] = data = dispatch_inspector(code, cursor_pos) or {}
        reply_content['metadata'] = {}
        reply_content['found'] = bool(data)
        if reply_content['found']: 
            if not kernel.shell.enable_html_pager:  reply_content['data'].pop('text/html', None)
            return reply_content
        return type(kernel).do_inspect(kernel, code, cursor_pos, 1)


In [12]:
    def load_ipython_extension(shell):
        shell.kernel.do_inspect = types.MethodType(do_inspect, shell.kernel)
        
    def unload_ipython_extension(shell):
        shell.kernel.do_inspect = types.MethodType(type(shell.kernel).do_inspect, shell.kernel)
    __name__ == '__main__' and load_ipython_extension(shell)